In [ ]:
import numpy as np
import msprime
import lshmm


### Input data


In [ ]:
# First, let's simulate some haplotypes to use as a reference panel.
seed = 1208
ts = msprime.sim_mutations(
    msprime.sim_ancestry(
        samples=10,
        ploidy=1,
        sequence_length=1e4,
        discrete_genome=True,
        recombination_rate=1e-4,
        random_seed=seed,
    ),
    rate=1e-4,
    random_seed=seed,
)
ts


In [ ]:
# A reference panel must be of size (number of sites, number of haplotypes).
ref_panel = ts.genotype_matrix()

# A query must be of size (1, number of sites).
# It can contain MISSING values, but not NONCOPY values.
query = ref_panel[:, 0].reshape(1, ts.num_sites)

# The number of distinct alleles per site is needed to get per-site mutation rates.
num_alleles = lshmm.core.get_num_alleles(ref_panel, query)


In [ ]:
# Per-site recombination and mutation probabilities are needed
# to parametrise the LS HMM.
rho = np.zeros(ts.num_sites, dtype=np.float64) + 1e-4
rho[0] = 0
mu = np.zeros(ts.num_sites, dtype=np.float64) + 1e-4


### Viterbi algorithm

In [ ]:
# Let's determine the best copying path and its log-likelihood using the Viterbi algorithm.
path, log_lik = lshmm.api.viterbi(
    ref_panel,
    query,
    num_alleles,
    prob_recombination=rho,
    prob_mutation=mu,
    scale_mutation_rate=True,
)
print(f"Parents in the path: {len(np.unique(path))}")
print(f"Log-likelihood: {log_lik}")


In [ ]:
# There is a function to evaluate the log-likelihood of a copying path.
# Let's check that we get the same log-likelihood of the same copying path 
# under the same parameters using the function.
log_lik_2 = lshmm.api.path_loglik(
    ref_panel,
    query,
    num_alleles,
    path,
    prob_recombination=rho,
    prob_mutation=mu,
    scale_mutation_rate=True,
)
print(f"Is the log-likelihood identical? {log_lik == log_lik_2}")


In [ ]:
# We should get a different log-likelihood value if the parameters are not the same.
# Let's check the log-likelihood of the same copying path under different parameters.
log_lik_3 = lshmm.api.path_loglik(
    ref_panel,
    query,
    num_alleles,
    path,
    prob_recombination=rho,
    prob_mutation=mu + 1e-5,    # Increase
    scale_mutation_rate=True,
)
print(f"Is the log-likelihood identical? {log_lik == log_lik_3}")


### Forward-backward algorithm

In [ ]:
# To calculate the forward probability matrix of a query,
# let's use the forward algorithm.
np.set_printoptions(linewidth=100, precision=4)
fwd_mat, norm_factor, log_lik = lshmm.api.forwards(
    ref_panel,
    query,
    num_alleles=num_alleles,
    prob_recombination=rho,
    prob_mutation=mu,
    scale_mutation_rate=True,
    normalise=True,
)
fwd_mat


In [ ]:
# Now, to calculate the backward probability matrix of the same query,
# let's use the backward algorithm.
np.set_printoptions(linewidth=100, precision=4)
bwd_mat = lshmm.api.backwards(
    ref_panel,
    query,
    num_alleles,
    normalisation_factor_from_forward=norm_factor,
    prob_recombination=rho,
    prob_mutation=mu,
    scale_mutation_rate=True,
)
bwd_mat


In [ ]:
# Check the integrity of the matrices.
np.allclose(np.sum(fwd_mat * bwd_mat, axis=1), 1.0)
